In [1]:
from itertools import permutations
import numpy as np
import math
from scipy.optimize import minimize

In [2]:
class Proportion:
    
    def __init__(self, v, f):
        self.value = v
        self.fraction = f

def get_composition(vals, fracs):
    return [Proportion(vals[i], fracs[i]) for i in range(len(vals))]
        
def get_elements(composition):
    return {composition[i].value: i for i in range(L)}

    
def generate_permutations(composition, N):
    arr = []
    for i in composition:
        arr.extend([i.value] * int(N*i.fraction)) # add fracs proportion of a species
    perms = permutations(arr, len(arr))
    return tuple(set(list(perms)))

def compute_hsum(h, composition, N):
    return np.sum(h * np.asarray([i.value*i.fraction for i in composition]) * N)

def energy_probabilities(permutations, elements, J, hsum):
    probs = np.array([0.] * len(permutations))
    for p in range(len(permutations)):
        perm = permutations[p]
        e = 0
        for i in range(len(perm)):
            cur, nex = perm[i], perm[(i+1)%len(perm)]
            jval = J[elements[cur]][elements[nex]]
            e += jval * cur * nex
            e += hsum
        probs[p] = math.exp(-e)
        
    probs = probs / np.sum(probs)
    return probs

def neg_entropy(probabilities):
    lg = np.vectorize(lambda x: math.log(x))
    return np.sum(probabilities * lg(probabilities))

def constraints(permutations, probabilities, elements, W, Lmd, test_con=False):
    gl_avgs = np.zeros((L, L))
    
    for p in range(len(permutations)):
        perm = permutations[p]
        avgs = np.zeros((L, L))
        for i in range(len(perm)):
            cur, nex = perm[i], perm[(i+1)%len(perm)]
            avgs[elements[cur]][elements[nex]] += cur * nex
        gl_avgs = gl_avgs + (avgs * probabilities[p])
        
    gl_avgs = (gl_avgs / len(permutations[0])) - W
    if test_con:
        print(gl_avgs)
    gl_avgs = gl_avgs.flatten()

    return np.sum(Lmd * np.vectorize(lambda x: x*x)(gl_avgs))
    

def lagrange_augmented(x, permutations, elements, composition, W, Lmd):
    h = x[:L]
    J = x[L:]
    J = J.reshape(L, L)
    hsum = compute_hsum(h, composition, len(permutations[0]))
    
    probs = energy_probabilities(permutations, elements, J, hsum)
    ent = neg_entropy(probs)
    con = constraints(permutations, probs, elements, W, Lmd)
    
    return ent + con

In [3]:
L = 2
N = 10
vals = [-1, 1]
fracs = [0.4, 0.6]
W = np.array([[0.9, 0.1], [0.1, 0.9]])
Lmd = np.array([1e3]*(L*L))
x = np.concatenate(([0.25]*2, [0.5]*4)) 

comp = get_composition(vals, fracs)
elems = get_elements(comp)
perms = generate_permutations(comp, N)
lagrange_augmented(x, perms, elems, comp, W, Lmd)

1589.2236071370178

In [4]:
m = minimize(lagrange_augmented, x, args=(perms, elems, comp, W, Lmd), options={'maxiter':1000, 'disp': True})
m

Optimization terminated successfully.
         Current function value: 597.697415
         Iterations: 16
         Function evaluations: 168
         Gradient evaluations: 21


      fun: 597.6974149702127
 hess_inv: array([[9.99755016e-01, 3.91396929e-03, 8.55044238e-01, 8.54986150e-01,
        8.54986150e-01, 8.55340079e-01],
       [3.91396929e-03, 1.00237219e+00, 2.54184240e+00, 2.53939561e+00,
        2.53939561e+00, 2.53362185e+00],
       [8.55044238e-01, 2.54184240e+00, 1.06357435e+03, 1.06192674e+03,
        1.06192674e+03, 1.06062724e+03],
       [8.54986150e-01, 2.53939561e+00, 1.06192674e+03, 1.06227931e+03,
        1.06127931e+03, 1.05998004e+03],
       [8.54986150e-01, 2.53939561e+00, 1.06192674e+03, 1.06127931e+03,
        1.06227931e+03, 1.05998004e+03],
       [8.55340079e-01, 2.53362185e+00, 1.06062724e+03, 1.05998004e+03,
        1.05998004e+03, 1.05968082e+03]])
      jac: array([-7.62939453e-06,  7.62939453e-06,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])
  message: 'Optimization terminated successfully.'
     nfev: 168
      nit: 16
     njev: 21
   status: 0
  success: True
        x: array([ 0.24549828,

In [5]:
h = m.x[:L]
J = m.x[L:]
J = [J[i*L:(i+1)*L] for i in range(L)]

hsum = compute_hsum(h, comp, len(perms[0]))
mprobs = energy_probabilities(perms, elems, J, hsum)
for i in range(len(mprobs)):
    print(perms[i], mprobs[i])

(1, 1, 1, 1, -1, -1, 1, 1, -1, -1) 3.306667672554392e-12
(-1, 1, 1, 1, -1, -1, 1, 1, 1, -1) 3.3066676725543686e-12
(-1, 1, -1, 1, 1, 1, 1, -1, -1, 1) 1.0934051099427825e-22
(1, -1, -1, 1, -1, 1, -1, 1, 1, 1) 1.093405109942771e-22
(1, -1, 1, 1, -1, 1, -1, 1, 1, -1) 3.6155273309501756e-33
(1, 1, -1, 1, 1, 1, -1, -1, 1, -1) 1.0934051099427785e-22
(1, 1, 1, 1, 1, -1, 1, -1, -1, -1) 3.3066676725543686e-12
(1, -1, 1, 1, -1, 1, 1, 1, -1, -1) 1.0934051099427865e-22
(-1, -1, 1, 1, 1, -1, 1, 1, 1, -1) 3.3066676725543686e-12
(1, -1, -1, -1, 1, 1, 1, -1, 1, 1) 3.3066676725543658e-12
(1, 1, 1, 1, 1, -1, -1, -1, -1, 1) 0.09999999997519968
(-1, 1, 1, 1, -1, 1, 1, -1, 1, -1) 1.0934051099427785e-22
(1, -1, 1, 1, 1, 1, -1, 1, -1, -1) 1.0934051099427785e-22
(1, 1, 1, -1, 1, -1, 1, -1, -1, 1) 1.0934051099427785e-22
(-1, 1, -1, -1, -1, 1, 1, 1, 1, 1) 3.3066676725543803e-12
(-1, 1, -1, -1, 1, 1, 1, 1, -1, 1) 1.0934051099427785e-22
(1, -1, -1, 1, 1, -1, 1, 1, -1, 1) 1.0934051099427785e-22
(-1, 1, 1, 1, 1, 1,

In [6]:
mcon = constraints(perms, mprobs, elems, W, Lmd, test_con=True)
mcon

[[-0.6 -0.2]
 [-0.2 -0.4]]


600.0000000694396